In [14]:
from sklearn.datasets import fetch_california_housing
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
import xgboost as xgb
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
import seaborn as sns

In [15]:
# Step 2: Load the Iris dataset
housing = fetch_california_housing()
df = pd.DataFrame(data=housing.data, columns=housing.feature_names)

#MedHouseVal: Median house value for households within a district (in $100,000s)

df['MedHouseVal'] = housing.target
print(df.shape)
df.head()

(20640, 9)


,MedInc,HouseAge,AveRooms,AveBedrms,Population,AveOccup,Latitude,Longitude,MedHouseVal
0,8.3252,41.0,6.984127,1.023810,322.0,2.555556,37.88,-122.23,4.526
1,8.3014,21.0,6.238137,0.971880,2401.0,2.109842,37.86,-122.22,3.585
2,7.2574,52.0,8.288136,1.073446,496.0,2.802260,37.85,-122.24,3.521
3,5.6431,52.0,5.817352,1.073059,558.0,2.547945,37.85,-122.25,3.413
4,3.8462,52.0,6.281853,1.081081,565.0,2.181467,37.85,-122.25,3.422


In [16]:
df.describe()

,MedInc,HouseAge,AveRooms,AveBedrms,Population,AveOccup,Latitude,Longitude,MedHouseVal
count,20640.000000,20640.000000,20640.000000,20640.000000,20640.000000,20640.000000,20640.000000,20640.000000,20640.000000
mean,3.870671,28.639486,5.429000,1.096675,1425.476744,3.070655,35.631861,-119.569704,2.068558
std,1.899822,12.585558,2.474173,0.473911,1132.462122,10.386050,2.135952,2.003532,1.153956
min,0.499900,1.000000,0.846154,0.333333,3.000000,0.692308,32.540000,-124.350000,0.149990
25%,2.563400,18.000000,4.440716,1.006079,787.000000,2.429741,33.930000,-121.800000,1.196000
50%,3.534800,29.000000,5.229129,1.048780,1166.000000,2.818116,34.260000,-118.490000,1.797000
75%,4.743250,37.000000,6.052381,1.099526,1725.000000,3.282261,37.710000,-118.010000,2.647250
max,15.000100,52.000000,141.909091,34.066667,35682.000000,1243.333333,41.950000,-114.310000,5.000010


In [17]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20640 entries, 0 to 20639
Data columns (total 9 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   MedInc       20640 non-null  float64
 1   HouseAge     20640 non-null  float64
 2   AveRooms     20640 non-null  float64
 3   AveBedrms    20640 non-null  float64
 4   Population   20640 non-null  float64
 5   AveOccup     20640 non-null  float64
 6   Latitude     20640 non-null  float64
 7   Longitude    20640 non-null  float64
 8   MedHouseVal  20640 non-null  float64
dtypes: float64(9)
memory usage: 1.4 MB


In [18]:
df.isnull().sum()

,0
MedInc,0
HouseAge,0
AveRooms,0
AveBedrms,0
Population,0
AveOccup,0
Latitude,0
Longitude,0
MedHouseVal,0


In [19]:
df.nunique()

,0
MedInc,12928
HouseAge,52
AveRooms,19392
AveBedrms,14233
Population,3888
AveOccup,18841
Latitude,862
Longitude,844
MedHouseVal,3842


In [20]:
#sns.pairplot(df, diag_kind='kde', plot_kws={'alpha':0.5, 's':30})
#plt.suptitle("Pairplot of California Housing Dataset", y=1.02)
#plt.show()

In [21]:
# prompt: remove outliers from df dataset

import numpy as np
from scipy import stats

# Calculate z-scores for all numerical columns
z_scores = stats.zscore(df.select_dtypes(include=['number']))

# Define a threshold for outlier detection (e.g., 3 standard deviations)
threshold = 3

# Filter the DataFrame to remove outliers
df_no_outliers = df[(np.abs(z_scores) < threshold).all(axis=1)]

print("Shape of original DataFrame:", df.shape)
print("Shape of DataFrame without outliers:", df_no_outliers.shape)


Shape of original DataFrame: (20640, 9)
Shape of DataFrame without outliers: (19794, 9)


In [22]:
X = df.drop('MedHouseVal', axis=1)
y = df['MedHouseVal']

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Initialize the Linear Regression model
model = LinearRegression()

# Train the model
model.fit(X_train, y_train)

# Make predictions on the test set
y_pred = model.predict(X_test)

# Evaluate the model
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f"Mean Squared Error (MSE): {mse}")
print(f"R² Score: {r2}")

Mean Squared Error (MSE): 0.5558915986952444
R² Score: 0.5757877060324508


In [23]:
X = df.drop('MedHouseVal', axis=1)
y = df['MedHouseVal']

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Initialize the XGBoost Regressor
model = xgb.XGBRegressor(
    objective ='reg:squarederror', # Objective function for regression
    n_estimators= 300, # Number of boosting rounds
    max_depth= 8, # Maximum tree depth for base learners
    learning_rate=0.2, # Step size shrinkage
    subsample=0.8, # Subsample ratio of the training instances
    colsample_bytree=0.8, # Subsample ratio of columns when constructing each tree
    random_state=42
)

# Train the model
model.fit(X_train, y_train)

# Make predictions on the test set
y_pred = model.predict(X_test)

# Evaluate the model
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f"Mean Squared Error (MSE): {mse}")
print(f"R² Score: {r2}")

Mean Squared Error (MSE): 0.20758918407308902
R² Score: 0.8415844308761087


In [27]:
result = pd.DataFrame({'Actual': y_test, 'Predicted': y_pred})
result["deviation"] = ((result["Predicted"] - result["Actual"])/result["Actual"])*100
print(result)

        Actual  Predicted  deviation
20046  0.47700   0.509813   6.879025
3024   0.45800   0.873375  90.693192
15663  5.00001   4.818850  -3.623201
20484  2.18600   2.315498   5.923965
9814   2.78000   2.305145 -17.081131
...        ...        ...        ...
15362  2.63300   2.547555  -3.245159
16623  2.66800   2.118652 -20.590260
18086  5.00001   4.957721  -0.845774
2144   0.72300   0.517437 -28.432011
3665   1.51500   1.533637   1.230192

[4128 rows x 3 columns]


In [25]:
from sklearn.metrics import mean_absolute_percentage_error
mape = mean_absolute_percentage_error(y_test, y_pred)
print(mape)

0.17229518044344053


In [26]:
testing_data = X_test.copy()
testing_data['MedHouseVal'] = y_test
testing_data['Predicted'] = y_pred
testing_data.to_csv("California house prediction.csv",index=0)